### Class ID to Name Mapping with RGB Values

In [101]:
class_id_to_name = {
    0:  ('road', [28, 42, 168]),
    1:  ('pool', [0, 50, 89]),
    2:  ('vegetation', [107, 142, 35]),
    3:  ('roof', [70, 70, 70]),
    4:  ('wall', [102, 102, 156]),
    5:  ('window', [254, 228, 12]),
    6:  ('person', [255, 22, 96]),
    7:  ('dog', [102, 51, 0]),
    8:  ('car', [9, 143, 150]),
    9:  ('bicycle', [119, 11, 32]),
    10: ('tree', [51, 51, 0]),
    11: ('truck', [160, 160, 60]),   # added truck
    12: ('bus', [200, 80, 80]),      # added bus
    13: ('vehicle', [20, 80, 80]),      # added bus
}

### Comparing YOLO Model Predictions on Video: Original vs Fine-Tuned


In [102]:
import cv2
from ultralytics import YOLO

def annotate_frame(frame, model, class_id_to_name, conf=0.25):
    """
    Annotates the frame with predictions from the model.
    """
    annotated = frame.copy()
    results = model(annotated, conf=conf, verbose=False)[0]

    if results.boxes is None or results.boxes.xyxy is None:
        return annotated  # No detections

    boxes = results.boxes.xyxy.cpu().numpy()
    class_ids = results.boxes.cls.cpu().numpy()
    confidences = results.boxes.conf.cpu().numpy()

    # Debugging: Print the number of detections and confidences
    # print(f"Model {model} detected {len(boxes)} boxes")
    # print(f"Confidences {model}: {confidences}")

    for box, cls_id, confidence in zip(boxes, class_ids, confidences):
        if confidence < conf:
            continue  # Skip detections below the confidence threshold
        x1, y1, x2, y2 = map(int, box)
        class_name, color = class_id_to_name.get(int(cls_id), (f"id:{int(cls_id)}", (0, 255, 255)))
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated, f"{class_name} {confidence:.2f}", (x1, max(y1 - 10, 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return annotated

def resize_to_height(image, target_height=420):
    """
    Resize image while maintaining aspect ratio.
    """
    h, w = image.shape[:2]
    scale = target_height / h
    new_w = int(w * scale)
    return cv2.resize(image, (new_w, target_height))

def compare_models_live(video_path, model_a_path, model_b_path, class_id_to_name, conf=0.25):
    """
    Compare the predictions of two models (Model A and Model B) on a video.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Failed to open video: {video_path}")
        return

    model_a = YOLO(model_a_path)
    model_b = YOLO(model_b_path)

    print("[INFO] Press 'q' to quit, 'p' to pause, 'r' to resume.")
    paused = False  # Variable to keep track of the pause state

    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("[INFO] End of video or failed to read frame.")
                break

            try:
                # Annotate frames using both models
                left = annotate_frame(frame, model_a, class_id_to_name, conf)
                right = annotate_frame(frame, model_b, class_id_to_name, conf)

                # Resize frames to a consistent height while maintaining the aspect ratio
                target_height = 420  # Target height for resizing
                left_resized = resize_to_height(left, target_height)
                right_resized = resize_to_height(right, target_height)

                # Find the maximum width between the two resized frames to align them horizontally
                combined_width = left_resized.shape[1] + right_resized.shape[1]
                combined = cv2.hconcat([left_resized, right_resized])

            except Exception as e:
                print(f"⚠️ Error during frame processing: {e}")
                continue

            # Display the combined output of both models
            cv2.imshow("Compare: Fine-Tuned (Left) vs Original (Right)", combined)

        # Key press handling
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # 'q' to quit
            break
        elif key == ord('p'):  # 'p' to pause
            paused = True
            print("[INFO] Paused. Press 'r' to resume.")
        elif key == ord('r'):  # 'r' to resume
            paused = False
            print("[INFO] Resumed.")

        # Ensure 'q' can still quit the program even if paused
        if paused:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):  # 'q' to quit
                break

    cap.release()
    cv2.destroyAllWindows()


### Find Best YOLO Model: Searching for 'best.pt' in Training Directory


In [103]:
from pathlib import Path

def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"✅ Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


## Comparing YOLO Models: Pretrained vs Fine-Tuned on Video

In [104]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

best_pt_path = find_best_model(old_path)
best_pt_path_retrain = find_best_model(new_path)


compare_models_live(
    video_path="./a/v1.mp4",
    model_a_path=best_pt_path,  # Pretrained model
    model_b_path=best_pt_path_retrain,  # Fine-tuned model
    class_id_to_name=class_id_to_name,
    conf=0.6
)

✅ Found best.pt at: runs\train\yolov8\weights\best.pt
✅ Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt
[INFO] Press 'q' to quit, 'p' to pause, 'r' to resume.
